# Модель "мешок слов"


Пусть есть коллекция документов $D$. Будем рассматривать модель *bag-of-words* (мешок слов), то есть каждый документ состоит из какого-то набора слов (терма) без учета их позиций внутри документа. 

Рассмотрим коллекцию документов:

In [1]:
docs = ['человек лев орел черепаха человек', 
        'лев вол орел',
        'лев черепаха лев кошка',
        'жучка кошка мышка',
        'лев орел грифон']

Посчитаем сколько каждое слово встретилось во всей коллекции и сколько каждом документе:

In [2]:
import re
import numpy as np
import pandas as pd


from collections import defaultdict, Counter 

def parse_doc(doc):
    return re.split(r'\s+', doc, re.U)

# подсчет df по коллекции
def calc_df_dict(docs):
    c = Counter() 
    for doc_id, doc in enumerate(docs):
        c.update(set(parse_doc(doc)))
    return c 
    
# подсчет tf для документа
def calc_tf_dict(doc):
    c = Counter() 
    for word in parse_doc(doc):
        c[word] += 1
    return c 
    
dfs = calc_df_dict(docs)
pd.DataFrame(data=list(dfs.items()), columns=['term', 'df'])

,term,df
0,черепаха,2
1,лев,4
2,человек,1
3,орел,3
4,вол,1
5,кошка,2
6,мышка,1
7,жучка,1
8,грифон,1


Каждому слову и каждому документу можно присвоить уникальный численный идентификатор, и построить так называемую tf-матрицу, состоящую из элементов $\{tf_{t,d}\}$ - вес слова $t$ в документе $d$ (где $t = 1 \dots n$ - индексы слов, $j = 1 \dots k$ - индексы документов). Под весом может подразумеваться число вхождений, нормализированная частота, и т.п.

### Система обозначений SMART

$\{tf_{t,d}\}$ не всегда дает адекватное представление (документы могут быть сильно не равной длины, не учитывается значимость слов). Поэтому существует более сложные модели под обобщенным названием *tf-idf* (term frequency - inverted document frequency), где используется документная частота слова. Идея в том, что для каждого слова в каждом документе считается *tf*, потом *idf* и значения перемножаются. Потом для каждого документа получается вектор, с которым можно что-то сделать (нормализовать).

| Частота термина                                                                                                  | Документная частота                                            | Нормировка                                              |
|------------------------------------------------------------------------------------------------------------------|----------------------------------------------------------------|---------------------------------------------------------|
| n $$	\text{tf}_{t,d}$$                                                                                             | n $$1$$                                                        | n $$1$$                                                 |
| l $$1 + \log{\text{tf}_{t,d}}$$                                                                                  | t $$\log{\frac{N}{\text{df}_i}}$$                              | c $$\frac{1}{\sqrt{\omega_1^2 + \ldots + \omega_m^2}}$$ |
| a $$0.5 + \frac{0.5 \text{tf}_{t,d}}{\max_t{ \text{tf}_{t,d}}}$$                                                 | p $$\text{max}(0, \log{\frac{N - \text{df}_i}{\text{df}_i}})$$ |
| b $$ 1,  \text{if }  \text{tf}_{t,d} > 0 \text{ else } 0 $$                                                              |
| L $$\frac{1 + \log{\text{tf}_{t,d}}}{1 + \log{\text{avg}_{t \in d}(\text{tf}_{t,d})}}$$                          |


### Ипользование scikit-learn

In [3]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer

vectorizer = CountVectorizer()
vectorizer.fit_transform(docs).toarray()  

array([[0, 0, 0, 0, 1, 0, 1, 2, 1],
       [1, 0, 0, 0, 1, 0, 1, 0, 0],
       [0, 0, 0, 1, 2, 0, 0, 0, 1],
       [0, 0, 1, 1, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 1, 0, 1, 0, 0]])

In [4]:
print('\n'.join(vectorizer.get_feature_names_out()))

вол
грифон
жучка
кошка
лев
мышка
орел
человек
черепаха


## Использование в кластеризации, классификации и тематическом моделировании

Напишем код для чтения и обработки коллекции новостей

In [5]:
import gzip

from dataclasses import dataclass
from typing import Iterator

from nltk.corpus import stopwords
from yargy.tokenizer import MorphTokenizer


@dataclass
class Text:
    label: str
    title: str
    text: str


def read_texts(fn: str) -> Iterator[Text]:
    with gzip.open(fn, "rt", encoding="utf-8") as f:
        for line in f:
            yield Text(*line.strip().split("\t"))


tokenizer = MorphTokenizer()
ru_stopwords = set(stopwords.words("russian"))


def normalize_text(text: str) -> str:
    tokens = [
        tok.normalized for tok in tokenizer(text) if tok.normalized not in ru_stopwords
    ]
    return " ".join(tokens)

In [6]:
normalize_text("Привет, миру!")

'привет , мир !'

Прочитаем текст и преобразуем документы в вектора

In [7]:
texts = list(read_texts("data/news.txt.gz"))

vectorizer = TfidfVectorizer(max_df=0.2, min_df=10)
# vectorizer = HashingVectorizer()

X = vectorizer.fit_transform([normalize_text(text.text) for text in texts]).toarray()

понизим размерность c помощью `PCA`

In [8]:
from sklearn.decomposition import PCA

pca = PCA(n_components=500)
X = pca.fit_transform(X)

In [9]:
X.shape

(10000, 500)

напишем кол для преобразования новых текстов

In [10]:
def transform_text(text: str) -> np.ndarray:
    normalized_text = normalize_text(text)
    vect = vectorizer.transform([normalized_text]).toarray()
    return pca.transform(vect)

transform_text("привет миру ещё раз")

array([[-3.39188448e-02, -7.66922681e-03, -7.87604724e-03,
         1.91158969e-03,  1.09120230e-02, -1.92687870e-02,
         8.63962326e-03,  1.66701983e-02,  6.38763014e-03,
        -1.73677798e-03,  1.03452095e-02, -3.54432869e-02,
        -4.87775038e-02, -1.62290528e-02, -2.66550769e-03,
         2.27416339e-02,  4.50016751e-03, -1.31262938e-03,
        -3.46951176e-03, -1.18351026e-02,  1.56695857e-02,
        -6.16916434e-03,  2.25223663e-02, -1.49859357e-02,
        -5.50202240e-03,  3.98797921e-02,  1.25061969e-02,
        -4.69093639e-03, -5.58295836e-03,  2.57390955e-03,
         8.86034321e-03, -1.55271465e-02, -7.09630521e-04,
        -2.31117436e-03, -7.23515606e-03,  6.70643369e-04,
        -3.04835396e-02, -2.85355520e-02,  8.21367025e-06,
         1.81021158e-03,  3.75946358e-03, -1.62730165e-02,
         9.20310085e-03,  4.11560649e-02, -4.69820532e-03,
        -3.52548460e-03, -6.88766878e-03,  2.56057847e-03,
         3.33875709e-02, -2.06323461e-02,  5.13790117e-0

Кластеризация

In [11]:
from sklearn.cluster import KMeans

k_means = KMeans(n_clusters=10, n_init="auto")
k_means.fit(X)

KMeans(n_clusters=10)

Классификация

In [12]:
from sklearn.svm import SVC

y = [text.label for text in texts]

svc = SVC()
svc.fit(X, y)

SVC()

In [13]:
svc.predict(transform_text("путешествие в Японию"))

array(['life'], dtype='<U9')

Тематическое моделирование

In [14]:
import re

from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel 

normalized_tokens = [
    re.findall(r"\b\w+\b", normalize_text(text.text))
    for text in texts
]
dictionary = Dictionary(normalized_tokens)

corpus = [dictionary.doc2bow(text) for text in normalized_tokens]

lda = LdaModel(corpus, num_topics=10)

In [15]:
id2token = {v : k for (k, v) in dictionary.token2id.items()}
[(id2token[token_id], p) for (token_id, p) in lda.get_topic_terms(6, 20)]

[('год', 0.02377364),
 ('который', 0.011447275),
 ('это', 0.0075186775),
 ('фильм', 0.0068119806),
 ('новый', 0.00609945),
 ('также', 0.0053507155),
 ('компания', 0.004517477),
 ('стать', 0.004180227),
 ('первый', 0.0036322556),
 ('картина', 0.003574336),
 ('работа', 0.003507077),
 ('время', 0.0034629167),
 ('свой', 0.0032433094),
 ('проект', 0.003137297),
 ('доллар', 0.0030117582),
 ('лента', 0.0030096709),
 ('сообщать', 0.0029000922),
 ('россия', 0.002763019),
 ('военный', 0.0027356558),
 ('получить', 0.00258979)]